###### Coursera  | Andrew Ng's Deep Learning Class | Course2. Improving Deep Neural Networks_Hyperparameter tuning, Regularization and Optimization | Week1. Practical aspects of Neural networks

이번주에는 하이퍼 파라미터 튜닝, 데이터를 어떻게 나눌 것인지, 최적화 알고리즘의 실행 속도를 증가시키는 방법, 정규화, 모형이 정상적으로 동작하는지 확인하는 방법 등 신경망 모형을 좀 더 잘 동작하게 만드는 실질적인 방법들에 대해 알아본다. 

# [1]. Setting up your Machine Learning application

### [1-1]. Train/dev/test sets

딥러닝 모형을 잘 동작하게 하려면 레이어의 수, 각 히든 레이어에 유닛을 몇개식 둘 것인지, learning rate은 얼마로 할 것인지, 활성함수로는 어떤 것을 사용할 것인지 등 결정해야 하는 요소들이 매우 많다. 새로운 모형에 대해 이런 요소들을 바로 잘 결정하는 건 거의 불가능한 일이라서 어떤 아이디어대로 이런 요소들을 정하고, 이것들을 구현하고, 데이터에 적용 해보고, 아이디어를 수정하여 다시 구현하고 적용하는 과정을 반복하면서 더 나은 방향을 찾는 방식이 일반적이다. 또한 특정 도메인에서 잘 적용되던 노하우가 다른 도메인에서는 크게 효과가 없는 경우가 많아서 이런 반복적인 과정이 필수적이다. (예를들어 NLP에서 잘 적용되던 방식이 비전 문제에는 효과가 없을 수도 있다.)

따라서 이런 반복 과정(아이디어 ⇒ 구현 ⇒ 실험 ⇒)을 얼마나 빠르고 효과적으로 많이 수행할 수 있느냐가 좋은 모형을 만드는데 매우 중요한 요소일수밖에 없다.

![Local image](./images/C2W1L01_01.PNG)

확보한 데이터를 일반적으로 **훈련데이터**(Training set), **개발데이터**(Hold-out cross validation set(=Development set)), **테스트데이터**(Test set)로 나눈다. 우선 훈련데이터로 모형 파라미터($W$, $b$..)를 학습시키고, 개발데이터로 (하이퍼 파라미터를 달리하는) 여러 모형중 성능이 나은 것을 찾고, 테스트데이터로 최종 모형에 대한 객관적인(unbiased) 모형의 성능을 측정한다.

과거의 전통적인 머신러닝에서는 전체 데이터를 6:2:2비율로 훈련데이터/개발데이터/테스트데이터로 나눠 사용하거나, 7:3 비율로 훈련데이터/개발데이터로 나누는 것이 일반적이었다. 그런데 이런 분할 비율은 전체 데이터가 몇백만건에서 몇만건 정도일 경우에는 합리적인 선택었다. 하지만 요즘과 같이 몇백만건 이상의 데이터를 다루는 것이 일반적인 상황에서는 예기가 달라진다. 사실 개발데이터는 여러 모형 중 어느것이 더 나은지만 빠르게 선택할 수 있는 정도면 충분하고, 모형의 객관적 성능 측정 또한 그리 많은 데이터가 필요하지 않다. 몇백만건의 데이터가 있을 경우 개발데이터와 테스트데이터에 몇만건 정도면 충분하며 이는 전체 데이터의 1%정도에 불과하다. 데이터가 더 많을 경우 전체 데이터의 0.5%이하 데이터 만으로도 충분히 개발데이터와 테스트데이터로 활용할 수 있다.

![Local image](./images/C2W1L01_02.PNG)

데이터를 구성하는데 있어 또 한가지 중요한 것은 훈련데이터와 개발/테스트 데이터의 분포가 같아야 한다는 점이다. 예를들어 고양이 그림을 구분하는 모형을 만든다고 할 때 훈련데이터는 사용자가 웹페이지에 업로드한 사진들을 사용하고, 개발/테스트데이터는 사용자가 모바일앱으로 올린 데이터를 사용하는 경우 두 데이터는 해상도/크기/구도 등 특성이 다를 것이기 때문에 훈련데이터로 학습한 모형을 개발/테이터가 잘 평가할 수 없을 것이다. 

그리고 모형 성능에 대한 객관적 지표(Unbiased estimate for the model performance)가 필요 없는 경우 테스트 데이터를 별도로 두지 않아도 된다. 이 경우 데이터는 훈련데이터와 개발데이터로 나뉠텐데, 이때의 개발데이터를 테스트데이터로 부르는 경우도 있으나 바람직한 명칭은 아니다. (하이퍼파라미터 튜닝과 모형 선택을 위해 사용되는 데이터를 개발데이터로 부르는 것을 권장한다.)

![Local image](./images/C2W1L01_03.PNG)

### [1-2]. Bias/Variance

Bias/Variance는 얼핏 보기에는 쉬운 것 같지만 사실 마스터하기는 간단치 않은 개념이며, 훌륭한 머신러닝 실무자들은 대부분 Bias/Variance를 아주 잘 이해하고 있다. 그만큼 정확한 이해가 중요한 개념이다.

잘 알다시피 머신러닝에 있어서 Bias/Variance를 다루는데 있어서 이 둘 사이에 절충점을 찾는 것(trade-off)이 중요하다. 그런데 딥러닝에서는 Bias를 해결하려는 것과 동시에 Variance도 해결하려 하는데 이 둘 사이의 trade-off에 대해서는 잘 논하지 않는다. 이 말이 무슨 뜻인지 차차 알아보도록 한다.

---

아래와 같이 O와 X로 구성된 이항 데이터가 있고 이들을 O와 X로 나누려는 어떤 선을 찾는다고 해 보자.

우선 첫번째 그림과 같은 직선을 사용하여 직선 위 점은 X로, 직선 아래 점들은 O라고 예측할 경우 상당히 많은 오-예측이 발생한다. 이런 경우를 'high bias' 혹은 under-fitting이라 한다. 반면 극단적으로 세번째 그림과 같이 주어진 데이터에 아주 잘 맞는 아주 복잡한 곡선을 사용할 경우의 상황을 'high variance' 혹은 over-fitting이라 한다. 이 두 경우의 절충형으로 두번째 그림과 같이 적당히 굴곡진 곡선을 사용할 경우 약간의 오차가 있지만 많은 경우 잘 들어맞는 예측을 할 수 있게 된다. 

데이터의 설명변수($x$, feature)가 둘뿐인 경우 이렇게 2차원상에 decision boundary를 표현할 수 있지만, 차원이 많아질 경우 시각화하는 것이 어려워진다.(물론 다차원 데이터를 시각화하기 위한 기법들이 있지만 2차원상에 표현하는 순간 많은 정보를 잃게 된다.)

![Local image](./images/C2W1L02_01.PNG)

(앞서 decision boundary를 시각화 방법외에) Bias/Variance 문제를 점검하기 위해 훈련데이터와 개발데이터에 대한 예측 에러를 확인하는 방법도 있다. 아래 그림과 같이 고양이 그림과 고양이가 아닌 그림들로 구성된 데이터가 있을 때 고양이 그림을 판별하는 모형을 만든다고 해 보자. 4가지 모형을 만들었고 훈련데이터와 개발데이터에 대한 에러는 아래 슬라이드와 같다. 

우선 첫번째 경우는 훈련데이터에 대해서는 아주 좋은 성능을 보이지만 개발데이터에 대해서는 성능이 상대적으로 좋지 않다. 이는 훈련데이터에 과적합(over-fitted)된 상태이고 잘 일반화(generalize)되지 않는 모형이다. 이런 경우 개발데이터 에러가 훈련 데이터 에러와 큰 차이를 보이는 상태를 표현하여 'high variance'라 한다.

두번째 경우는 훈련데이터에 대해서 성능이 좋지 않고, 개발데이터에 대해서는 훈련데이터에 비해 성능 차이가 크지 않지만 이 또한 성능이 좋지 않다. 이 모형은 훈련데이터에 부적합(underfitting)하며 개발데이터에 일반화(generalize)는 잘 된다고 할 수 있다. 이런 경우 훈련 데이터에러가 (기대보다) 큰 상황을 표현하여 'high bias'라 한다.

세번째 경우 훈련데이터 에러가 높고, 개발데이터 에러는 이와 큰 차이로 더 높은 상황이다. 이 경우 두번째 경우와 마찬가지로 훈련데이터 에러가 크므로 'high bias'이고, 첫번째 경우와 마찬가지로 훈련데이터 에러와 테스트 데이터 에러가 큰 상황이므로 'high variance'라 한다. 이는 최악의 상황이다.

네번재 경우 훈련데이터 에러도 낮고, 이와 작은 차이로 개발데이터 에러가 낮으므로 'low bias', 'low variance'라 한다. 이는 아주 바람직한 상황이라 할 수 있다.

이런 논의에서 훈련데이터 에러가 높은지 낮은지에 대한 판단 기준은 어떻게 정한 것일까? 일반적으로 사람이 이런 그림을 판별할 경우 거의 에러가 없을 것이므로 훈련데이터에 대한 기대 에러 또한 매우 낮을 것이고 이 수준(0%에러)를 optimal error 혹은 bayes error라 하고 이 기대 수준과 훈련 데이터 에러를 비교하여 'bias'가 큰지 작은지를 판단한다. 만약 확보한 데이터가 찌그러지고 흐린 이미지들이라면 사람이라도 그것들을 판별하는 것이 쉽지 않을 것이고 bayes error로 0%가 아니라 5% 혹은 13%로 설정하고 이 경우 훈련데이터에 대한 15%에러 나쁜 결과가 아닐 수 있다. 

결국 기대수준(optimal error)과 훈련데이터 에러를 비교하여 high bias 문제가 있는지를 판단하며, 훈련데이터 에러와 개발데이터 에러를 비교하여 high variance 문제가 있는지를 판단한다.

![Local image](./images/C2W1L02_02.PNG)

앞서 high bias인 상황(직선)과 high variance(매우 복잡한 곡선)인 상황을 decision boundary를 그려 확인할 수 있었다. 그렇다면 위 세번째 상황과 같이 high bias & high variance인 상황의 decision boundary는 어떤 모습일까? 

아래 그림과 같이 직선과 중앙의 곡선이 혼합된 형태를 생각해볼 수 있고, 직선의 부적합 혹은 무딘 판별력이 high bias를 곡선의 과적합 혹은 잘 일반화되지 않는 특성이 high variance를 야기한다.

![Local image](./images/C2W1L02_03.PNG)

### [1-3]. Basic "recipe" for machine learning

.

![Local image](./images/C2W1L02b_01.PNG)

.

![Local image](./images/C2W1L02b_02.PNG)

.

# [2]. Regularizing your neural network

### [2-1]. Regularization

.

![Local image](./images/C2W1L03_01.PNG)

.

![Local image](./images/C2W1L03_02.PNG)

.

![Local image](./images/C2W1L03_03.PNG)

.

![Local image](./images/C2W1L03_04.PNG)

.

![Local image](./images/C2W1L03_05.PNG)

.

.

### [2-2]. Why regularization reduces overfitting

.

![Local image](./images/C2W1L03b_01.PNG)

.

![Local image](./images/C2W1L03b_02.PNG)

.

.

### [2-3] Dropout regularization

.

![Local image](./images/C2W1L04_01.PNG)

.

![Local image](./images/C2W1L04_02.PNG)

.

![Local image](./images/C2W1L04_03.PNG)

.

.

### [2-4] Understanding dropout

.

![Local image](./images/C2W1L04b_01.PNG)

.

.

### [2-5]. Other regularization methods

.

![Local image](./images/C2W1L05_01.PNG)

.

![Local image](./images/C2W1L05_02.PNG)

.

# [3]. Setting up your optimization problem

### [3-1]. Normalizing inputs

.

![Local image](./images/C2W1L06_01.PNG)

.

![Local image](./images/C2W1L06_02.PNG)

.

### [3-2]. Vanishing/exploding gradients

.

.

![Local image](./images/C2W1L07_01.PNG)

.

### [3-3]. Weight initialization for deep networks

.

![Local image](./images/C2W1L07_02.PNG)

.

### [3-4]. Numerical approximation of gradients

.

![Local image](./images/C2W1L08_01.PNG)

.

![Local image](./images/C2W1L08_02.PNG)

.

### [3-5]. Gradient Checking

.

![Local image](./images/C2W1L09_01.PNG)

.

![Local image](./images/C2W1L09_02.PNG)

.

### [3-6]. Gradient Checking implementation notes

.

![Local image](./images/C2W1L09b_01.PNG)

.

.

.